In [1]:
import numpy as np

In [2]:
blue_color = np.array([0.5, 0.7, 1.0])
white_color = np.array([1.0, 1.0, 1.0])

def length(v) :
    s = 0
    for x in v :
        s = s + x * x
    return np.sqrt(s)

def normalize(v) :
    vec_length = length(v)
    return v / vec_length

def color(dir) :
    unit_dir = normalize(dir)
    t = ( unit_dir[1] + 1.0 ) * 0.5
    return white_color * (1.0 - t) + blue_color * t

In [3]:
width  = 200
height = 100
img_name = "2.ray-casting.ppm"

origin_pos = np.array([0.0, 0.0, 0.0])
left_bottom = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])

with open(img_name, 'w') as img :
    img.write("P3\n{0:d} {1:d}\n255\n".format(width, height))
    
    for j in range(height - 1, -1, -1) :
        for i in range(0, width, 1) :
            u = float(i) / float(width )
            v = float(j) / float(height)
            
            target_pos = left_bottom + horizontal * u + vertical * v
            direction = target_pos - origin_pos
            
            target_color = color(direction)
            ir = int(target_color[0] * 255)
            ig = int(target_color[1] * 255)
            ib = int(target_color[2] * 255)
            
            img.write("{0:d} {1:d} {2:d}\n".format(ir, ig, ib))

In [4]:
def hit_sphere(center, radius, ray_origin, ray_direction) :
    oc = ray_origin - center
    a = np.dot(ray_direction, ray_direction)
    b = 2.0 * np.dot(ray_direction, oc)
    c = np.dot(oc, oc) - radius * radius
    
    D = b * b - 4 * a * c
    return D > 0

def color_sphere(ray_origin, ray_direction) :
    sphere_center = np.array([0, 0, -1])
    sphere_radius = 0.5
    if (hit_sphere(sphere_center, sphere_radius, ray_origin, ray_direction)) :
        return np.array([0 ,1, 0])
    else :
        return color(ray_direction)    

In [5]:
width  = 200
height = 100
img_name = "3.ray-casting-sphere.ppm"

origin_pos = np.array([0.0, 0.0, 0.0])
left_bottom = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])

with open(img_name, 'w') as img :
    img.write("P3\n{0:d} {1:d}\n255\n".format(width, height))
    
    for j in range(height - 1, -1, -1) :
        for i in range(0, width, 1) :
            u = float(i) / float(width )
            v = float(j) / float(height)
            
            target_pos = left_bottom + horizontal * u + vertical * v
            direction = target_pos - origin_pos
            
            target_color = color_sphere(origin_pos, direction)
            ir = int(target_color[0] * 255)
            ig = int(target_color[1] * 255)
            ib = int(target_color[2] * 255)
            
            img.write("{0:d} {1:d} {2:d}\n".format(ir, ig, ib))

In [8]:
def hit_sphere_normal(center, radius, ray_origin, ray_direction) :
    oc = ray_origin - center
    a = np.dot(ray_direction, ray_direction)
    b = 2.0 * np.dot(ray_direction, oc)
    c = np.dot(oc, oc) - radius * radius
    
    D = b * b - 4 * a * c
    if D < 0 :
        return -1.0
    else :
        return (-b - np.sqrt(D) ) / (2.0 * a)

def color_sphere_normal(ray_origin, ray_direction) :
    sphere_center = np.array([0, 0, -1])
    sphere_radius = 0.5
    t = hit_sphere_normal(sphere_center, sphere_radius, ray_origin, ray_direction)
    if t > 0.0 :
        n = normalize( ray_origin + t * ray_direction - np.array([0, 0, -1]))
        return 0.5 * (n + np.array([1.0, 1.0, 1.0]))
    else :
        return color(ray_direction)    

In [9]:
width  = 200
height = 100
img_name = "4.ray-casting-sphere.ppm"

origin_pos = np.array([0.0, 0.0, 0.0])
left_bottom = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])

with open(img_name, 'w') as img :
    img.write("P3\n{0:d} {1:d}\n255\n".format(width, height))
    
    for j in range(height - 1, -1, -1) :
        for i in range(0, width, 1) :
            u = float(i) / float(width )
            v = float(j) / float(height)
            
            target_pos = left_bottom + horizontal * u + vertical * v
            direction = target_pos - origin_pos
            
            target_color = color_sphere_normal(origin_pos, direction)
            ir = int(target_color[0] * 255)
            ig = int(target_color[1] * 255)
            ib = int(target_color[2] * 255)
            
            img.write("{0:d} {1:d} {2:d}\n".format(ir, ig, ib))

In [24]:
def hit_sphere_normal_many(sphere_list, ray_origin, ray_direction) :
    num_sphere = len(sphere_list)
    t = -1.0
    
    for i in range(num_sphere) :
        sphere_center = sphere_list[i][0]
        sphere_radius = sphere_list[i][1]
        
        oc = ray_origin - sphere_center
        a = np.dot(ray_direction, ray_direction)
        b = 2.0 * np.dot(ray_direction, oc)
        c = np.dot(oc, oc) - sphere_radius * sphere_radius
        
        D = b * b - 4 * a * c
        temp_t = (-b - np.sqrt(D) ) / (2.0 * a)
        if temp_t >= t :
            t = temp_t
        
    return t

def color_sphere_normal_many(sphere_list, ray_origin, ray_direction) :
    t = hit_sphere_normal_many(sphere_list, ray_origin, ray_direction)
    if t > 0.0 :
        n = normalize( ray_origin + t * ray_direction - np.array([0, 0, -1]))
        return 0.5 * (n + np.array([1.0, 1.0, 1.0]))
    else :
        return color(ray_direction)    

In [30]:
width  = 200
height = 100
img_name = "5.ray-casting-many_sphere.ppm"

origin_pos = np.array([0.0, 0.0, 0.0])
left_bottom = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])

sphere_list = [ 
    [ np.array([0, 0, -1]), 0.5 ],
    [ np.array([0.6, 0.6, -1.5]), 0.6 ],
    [ np.array([0.6, 0, -1.3]), 0.9 ]
]

with open(img_name, 'w') as img :
    img.write("P3\n{0:d} {1:d}\n255\n".format(width, height))
    
    for j in range(height - 1, -1, -1) :
        for i in range(0, width, 1) :
            u = float(i) / float(width )
            v = float(j) / float(height)
            
            target_pos = left_bottom + horizontal * u + vertical * v
            direction = target_pos - origin_pos
            
            target_color = color_sphere_normal_many(sphere_list, origin_pos, direction)
            ir = int(target_color[0] * 255)
            ig = int(target_color[1] * 255)
            ib = int(target_color[2] * 255)
            
            img.write("{0:d} {1:d} {2:d}\n".format(ir, ig, ib))

c:\users\administrator\desktop\github\pvenv\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
